# Backpack

In [1]:
from transformers import AutoTokenizer
from transformers import BertTokenizer
import torch
import os
import numpy as np
import torch.nn.functional as F
from scipy.stats import spearmanr

In [2]:
model = torch.load("/home/hamvir/NLP/Project/dataset/bert/results_2000/model.pt", map_location=torch.device('cpu'))  # You can specify the device (e.g., 'cuda:0') if using GPU

In [ ]:
model.eval()

In [4]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [5]:
def simm(w1,w2):
    x1 = tokenizer(w1,return_tensors='pt',add_special_tokens=False)['input_ids']
    x1 = model.backpack.word_embeddings(x1)
    x1 = model.backpack.sense_network(x1).detach()
    x1 = x1[:,:,0,:]
    x1 = x1.squeeze()
    x1 = F.normalize(x1,p=2,dim=1)
    x2 = tokenizer(w2,return_tensors='pt',add_special_tokens=False)['input_ids']
    x2 = model.backpack.word_embeddings(x2)
    x2 = model.backpack.sense_network(x2).detach()
    x2 = x2[:,:,0,:]
    x2 = x2.squeeze()
    x2 = F.normalize(x2,p=2,dim=1)
    sim_list = torch.sum(torch.multiply(x1,x2),dim=1)
    #print(sim_list)
    return(sim_list.min())

## SIMVERB

In [6]:
word1 = []
word2 = []  # add space
h_score = []
m_score =[]

In [7]:
with open('/home/hamvir/NLP/Project/dataset/Hamvir/gpt_2_trained/SimVerb-3500.txt', 'r') as file:
    # Iterate through each line
    for line in file:
        # Split the line into columns
        columns = line.strip().split('\t')

        # Extract word1, word2, and the score
        word1.append(" "+ columns[0])
        word2.append(" "+ columns[1])
        h_score.append(float(columns[3]))  # Assuming the score is a floating-point number


In [8]:
for i in range(len(word1)):
    m_score.append(simm(word1[i],word2[i]))

In [9]:
spearman_corr, _ = spearmanr(h_score, m_score)
spearman_corr

0.0868790515236678

## SIMMLEX

In [10]:
word1 = []
word2 = []  # add space
h_score = []
m_score =[]

In [11]:
with open('/home/hamvir/NLP/Project/dataset/Hamvir/gpt_2_trained/SimLex-999.txt', 'r') as file:
    # Iterate through each line
    header = next(file)
    for line in file:
        # Split the line into columns
        columns = line.strip().split('\t')

        # Extract word1, word2, and the score
        word1.append(" "+ columns[0])
        word2.append(" "+ columns[1])
        h_score.append(float(columns[3]))  # Assuming the score is a floating-point number


In [12]:
for i in range(len(word1)):
    m_score.append(simm(word1[i],word2[i]))

In [14]:
spearman_corr, _ = spearmanr(h_score, m_score)
spearman_corr

0.18058067159759492

# Bert

In [41]:
from transformers import BertConfig, BertModel
configuration = BertConfig()
model = BertModel(configuration)

In [72]:
from transformers import BertForMaskedLM, BertTokenizer

# Load pre-trained BERT model
model_name = 'bert-base-uncased'  # Example model name, you can use other variants
model = BertForMaskedLM.from_pretrained(model_name)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
model.eval()

In [74]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [79]:
def simm(w1,w2):
    x1 = tokenizer(w1,return_tensors='pt',add_special_tokens=False)['input_ids']
    x1 = model.bert.embeddings.word_embeddings(x1).detach() # (1, 1, 768)
    #print(x1.shape)
    x1 = x1[:,-1,:]  # (1,768)
    #print(x1.shape)
    x1 = F.normalize(x1,p=2,dim=1)
    x2 = tokenizer(w2,return_tensors='pt',add_special_tokens=False)['input_ids']
    x2 = model.bert.embeddings.word_embeddings(x2).detach() # (1, 1, 768)
    x2 = x2[:,-1,:] # (1,768)
    #print(x2.shape)
    x2 = F.normalize(x2,p=2,dim=1)
    sim_list = torch.sum(torch.multiply(x1,x2))
    #print(sim_list)
    return(sim_list.item())

## SIMVERB

In [80]:
word1 = []
word2 = []  # add space
h_score = []
m_score =[]

In [81]:
with open('/home/hamvir/NLP/Project/dataset/Hamvir/gpt_2_trained/SimVerb-3500.txt', 'r') as file:
    # Iterate through each line
    for line in file:
        # Split the line into columns
        columns = line.strip().split('\t')

        # Extract word1, word2, and the score
        word1.append(" "+ columns[0])
        word2.append(" "+ columns[1])
        h_score.append(float(columns[3]))  # Assuming the score is a floating-point number


In [82]:
for i in range(len(word1)):
    m_score.append(simm(word1[i],word2[i]))

In [83]:
spearman_corr, _ = spearmanr(h_score, m_score)
spearman_corr

0.2455796022923943

## SIMMLEX

In [84]:
word1 = []
word2 = []  # add space
h_score = []
m_score =[]

In [85]:
with open('/home/hamvir/NLP/Project/dataset/Hamvir/gpt_2_trained/SimLex-999.txt', 'r') as file:
    # Iterate through each line
    header = next(file)
    for line in file:
        # Split the line into columns
        columns = line.strip().split('\t')

        # Extract word1, word2, and the score
        word1.append(" "+ columns[0])
        word2.append(" "+ columns[1])
        h_score.append(float(columns[3]))  # Assuming the score is a floating-point number


In [86]:
for i in range(len(word1)):
    m_score.append(simm(word1[i],word2[i]))

In [87]:
spearman_corr, _ = spearmanr(h_score, m_score)
spearman_corr

0.46547584403104386